In [19]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.layers import Dense, Input
import os
import tensorflow as tf

这段代码实现了一个基于深度 Q 网络（DQN）的智能体类，用于强化学习任务。智能体通过与环境交互，存储经验数据，并使用这些数据进行模型训练和更新，以学习最优的决策策略。同时，还提供了模型的保存和加载功能，以及可视化模型结构的操作。

In [14]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        """
        初始化 DQNAgent 类的实例

        :param state_size: 环境状态的维度大小
        :param action_size: 动作空间的大小，即可能的动作数量
        """
        self.state_size = state_size
        self.action_size = action_size
        # 创建一个双端队列作为记忆库，最大长度为 2000，用于存储智能体的经验
        self.memory = deque(maxlen=2000)
        # 折扣因子，用于计算未来奖励的折扣值，0.95 表示未来奖励的重要性相对当前奖励有所降低
        self.gamma = 0.95
        # 探索率，初始值为 0.4，表示智能体有 40%的概率进行随机探索
        self.epsilon = 0.4
        self.epsilon_min = 0.01  # 探索率的最小值，当探索率降低到这个值时，不再继续降低
        self.epsilon_decay = 0.995  # 探索率的衰减率，每次更新后，探索率乘以这个值
        self.learning_rate = 0.001  # 模型学习率，用于调整模型权重的更新步长

        # 构建深度 Q 网络模型
        self.model = self._build_model()
        # 可视化 MLP 结构，并将可视化结果保存为 'dqn-cartpole-v0-mlp.png' 文件，不显示形状信息
        plot_model(self.model, to_file='dqn-cartpole-v0-mlp.png', show_shapes=False)

    def _build_model(self):
        """
        构建深度 Q 学习模型

        :return: 构建好的 Keras 模型
        """
        # 创建一个顺序模型
        model = Sequential()
        # 添加输入层，输入形状为 (state_size,)，即根据环境状态的维度确定输入形状
        model.add(Input(shape=(self.state_size,)))
        # 添加第一个隐藏层，有 24 个神经元，使用 relu 激活函数
        model.add(Dense(24, activation='relu'))
        # 添加第二个隐藏层，同样有 24 个神经元，使用 relu 激活函数
        model.add(Dense(24, activation='relu'))
        # 添加输出层，神经元数量为 action_size，使用线性激活函数，输出每个动作的 Q 值估计
        model.add(Dense(self.action_size, activation='linear'))
        # 编译模型，使用均方误差（MSE）作为损失函数，Adam 优化器，并传入学习率
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        """
        将智能体的经验（状态、动作、奖励、下一个状态、是否结束）存储到记忆库中

        :param state: 当前状态
        :param action: 采取的动作
        :param reward: 获得的奖励
        :param next_state: 下一个状态
        :param done: 是否完成当前 episode
        """
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        """
        根据当前状态决定智能体的动作

        :param state: 当前状态
        :return: 智能体要采取的动作
        """
        # 以 epsilon 的概率进行随机探索，选择一个随机动作
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        # 否则，使用模型预测当前状态下每个动作的 Q 值，并选择 Q 值最大的动作
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        """
        从记忆库中随机采样一批经验数据，并进行模型训练更新

        :param batch_size: 每次采样的经验数据数量
        """
        # 从记忆库中随机抽取 batch_size 个经验数据
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            # 计算目标 Q 值，如果 episode 未结束，则目标 Q 值为当前奖励加上未来最大 Q 值的折扣值
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            # 预测当前状态下的 Q 值
            target_f = self.model.predict(state)
            # 更新目标 Q 值中采取的动作对应的 Q 值为计算得到的目标 Q 值
            target_f[0][action] = target
            # 使用更新后的目标 Q 值对模型进行一次训练，不显示训练过程的详细信息
            self.model.fit(state, target_f, epochs=1, verbose=0)
        # 如果探索率大于最小值，则按照衰减率进行衰减
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        """
        加载预训练的模型权重

        :param name: 权重文件的名称
        """
        self.model.load_weights(name)

    def save(self, name):
        """
        保存当前模型的权重

        :param name: 保存权重的文件名称
        """
        self.model.save_weights(name)

In [20]:
env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

In [16]:
state_size,action_size

(4, 2)

In [21]:
agent = DQNAgent(state_size, action_size)

done = False
batch_size = 32
avg=0

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

EPISODES = 500

for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}".format(e, EPISODES, time, agent.epsilon))
            avg+=time
            break
    if len(agent.memory) > batch_size:
      agent.replay(batch_size)

Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
episode: 204/500, score: 43, e: 0.14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [ ]:
"Avg score:{}".format(avg/1000)